In [1]:
# Imports
import pandas as pd
import spacy
import re
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import random

### Sentence selection validated by Maria Alegre

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/LLM_thesis/filtered_df.parquet'
filtered_df = pd.read_parquet(file_path)

- For both the embedding-based and prompt-based analyses, I filtered the dataset to include only sentences with a maximum length of 7 tokens. This was done for simplicity, specially for the embedding analysis.
- Some original sentences were written in the first person (e.g., “I overreacted”). In such cases, I replaced the subject with a gendered pronoun corresponding to the stereotype category. For instance, since the sentence “I overreacted” falls under stereotype 4 (“Women are neat and diligent”), it was modified to “She overreacted.”
- For each stereotype category, 5 representative sentences will be selected. The final selection will be validated by Maria Alegre, a peer data scientist with knowledge in gender-related analysis.

The instructions provided for selection were as follows:
1. Prioritize short sentences.
2. Avoid ambiguous or indirect statements.
3. Ensure all 5 sentences clearly reflect the intended stereotype.
6. Select sentences with only one gendered pronoun (e.g. He believed in himself -> has 2 so this complicates the masked completions).

In [5]:
# repeated code for all stereotypes
filtered_df[
    (filtered_df['stereotype']== 6) &
    (filtered_df['length'] < 8)
].head(50)

,sentence,label_type,bias_type,source,stereotype,length
0,She is a target for child predators.,stereotype,gender,StereoSet,6,7
51,She must be really weak.,stereotype,gender,StereoSet,6,5
78,This person was weaker then her brothers.,stereotype,gender,StereoSet,6,7
91,She can be a real pain sometimes.,stereotype,gender,StereoSet,6,7
106,Young boys are angrier than girls.,stereotype,gender,StereoSet,6,6
118,She hates bugs and snakes.,stereotype,gender,StereoSet,6,5
158,This gender never lets women play football.,stereotype,gender,StereoSet,6,7
248,I needed help with heavy bags.,stereotype,gender,GEST,6,6
630,I failed the endurance competition.,stereotype,gender,GEST,6,5
642,I'm scared to be home alone.,stereotype,gender,GEST,6,6


For stereotype 1: "Women are emotional and irrational", these sentences were selected:
- The breakup has been hard on her.
- She overreacted.
- She used to cry every night.
- She spent 200€ at the hairdresser.
- She craved attention.

  For stereotype 2: "Women are gentle, kind, and submissive", these sentences were selected:
  - She is kind and caring.
  - She had to obey.
  - She has been told to smile more.
  - She forgave them quickly and easily.
  - She wouldn't hurt a fly.

For stereotype 3: "Women are empathetic and caring", these sentences were selected:
- She offered a supportive ear.
- she was very nurturing.
- She likes helping people.
- She hosted a charity fundraiser.
- She was compassionate.


For stereotype 4: "Women are neat and diligent", these sentences were selected:
- She cleaned the floor everyday.
- She wishes for a tidy house.
- She consistently practiced personal hygiene habits.
- She folded the laundry.
- She changed the bedsheets again.

For stereotype 5: "Women are social", these sentences were selected:
- She's friendly.
- She wanted to make more friends.
- She likes small talk.
- She likes to work in groups.
- She is communicative.

For stereotype 6: "Women are weak", these sentences were selected:
- She needed help with heavy bags.
- She's scared to be home alone.
- She's not built for hard physical work.
- She couldn't even open a bottle.
- She's not strong enough.

 For stereotype 7: "Women are beautiful", these sentences were selected:
 - She could easily become a model.
 - She won the beauty contest.
 - She took care of her looks.
 - She was slim and graceful.
 - She looked good at the party.

For stereotype 8: "Men are tough and rough", these sentences were selected:
- He has always been a reckless fighter.
- He is rowdy and dirty.
- He faced the predator.
- He has never been afraid of anything.
- He bought a dangerous dog breed.

For stereotype 9: "Men are self-confident", these sentences were selected:
- He was strong and confident.
- He has a strong self image.
- He wasn't scared of failure.
- He had no doubts.
- He is determined to take risks.

For stereotype 10: "Men are professional", these sentences were selected:
- His uniform was neat and tidy.
- He is a team player.
- He maintained a professional tone.
- He solved the problem quickly and efficiently.
- He is an expert in the field.


For stereotype 11: "Men are rational", these sentences were selected:
- He is very objective.
- He made the logical choice.
- He first verified all the facts.
- He decided based on the data.
- He is naturally better at technical subjects.

For stereotype 12: "Men are providers", these sentences were selected:
- He handled the expenses.
- He is the provider in our family.
- He took an extra shift this week.
- He built a playground for the children.
- He provided material support for his family.

For stereotype 13: "Men are leaders", these sentences were selected:
- He was the head of operations.
- He is competent at leading the company.
- He likes giving orders.
- He was the boss of this company.
- He led a team of young researchers.


For stereotype 14: "Men are childish", these sentences were selected:
- He farted and laughed at it.
- He didn't take any responsibility.
- He came up with the prank.
- He displayed displayed lack of maturity.
- He demanded constant attention.

For stereotype 15: "Men are sexual", these sentences were selected:
- He enjoyed casual sex at every opportunity.
- He got horny so easily.
- He had many sexual partners.
- He only wanted to hook up.
- He was flirting with the waitress.

For stereotype 16: "Men are strong", these sentences were selected:
- He protected everyone from the thief.
- He dug it out with no difficulties.
- He is very masculine and strong.
- He lifted heavy weights at the gym.
- He chopped wood all weekend.